In [1]:
import csv
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

sk = []
kr = []
uid = []
peak = []
duration = []
freq = []
high_avg = []

df = pd.read_csv('training_set_0s_30.csv')
df = df.append(pd.read_csv('training_set_1s_30.csv'))
#print(df)

df = df.groupby('id', as_index=False, sort=False)['ev', 'skewness', 'kurtosis', 'peak_usage', 
                                                  'avg_high_energy_usage_duration', 
                                                  'high_energy_usage_frequency', 'high_energy_usage_avg'].mean()
#testing...
df.to_csv('aggregate_training_30min.csv')
uid = np.array(df['ev'].values)
sk = np.array(df['skewness'].values)
kr = np.array(df['kurtosis'].values)
peak = np.array(df['peak_usage'].values)
duration = np.array(df['avg_high_energy_usage_duration'].values)
freq = np.array(df['high_energy_usage_frequency'].values)
high_avg = np.array(df['high_energy_usage_avg'].values)

#normalization
#sk_norm = sk/np.linalg.norm(sk, ord=np.inf, axis=0, keepdims=True)
#kr_norm = kr/np.linalg.norm(kr, ord=np.inf, axis=0, keepdims=True)
#peak_norm = peak/np.linalg.norm(peak, ord=np.inf, axis=0, keepdims=True)
#duration_norm = duration/np.linalg.norm(duration, ord=np.inf, axis=0, keepdims=True)
#freq_norm = freq/np.linalg.norm(freq, ord=np.inf, axis=0, keepdims=True)
#high_avg_norm = high_avg/np.linalg.norm(high_avg, ord=np.inf, axis=0, keepdims=True)
#kr_norm = sklearn.preprocessing.normalize(kr)

X = []
with open('training_set_norm.csv', 'w') as fou:
    #w = csv.writer(fou, delimiter=',', lineterminator='\n')
    #w.writerow(['ev_id', 'skew_norm', 'kurtosis_norm', 'peak_norm', 'duration_norm', 'freq_norm', 'high_avg_norm'])
    #for i in range(0, len(uid)):
    #    X.append([sk_norm[i], kr_norm[i], high_avg_norm[i], peak_norm[i]])
    #    w.writerow([uid[i], sk_norm[i], kr_norm[i], peak_norm[i], duration_norm[i], freq_norm[i], high_avg_norm[i]])
    w = csv.writer(fou, delimiter=',', lineterminator='\n')
    w.writerow(['ev_id', 'skew_norm', 'kurtosis_norm', 'peak_norm', 'duration_norm', 'freq_norm', 'high_avg_norm'])
    for i in range(0, len(uid)):
        X.append([sk[i], kr[i], high_avg[i], peak[i], duration[i], freq[i]])
        w.writerow([uid[i], sk[i], kr[i], peak[i], duration[i], freq[i], high_avg[i]])
#, peak_norm[i],duration_norm[i],freq_norm[i],high_avg_norm[i]        

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier


Y = uid
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
from joblib import dump, load
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
dump(scaler, 'scaler_30.joblib')
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)
clf1 = SVC(gamma='scale', class_weight='balanced', kernel='rbf', shrinking=True, probability=True)
#print(Y)
clf1.fit(X_train_transformed, Y_train)
#print(clf.support_vectors_)
#print(clf.n_support_) 

dump(clf1, 'trained_ev_detect_svm_30.joblib') 
test_Y = clf1.predict(X_test_transformed)

#clf = LogisticRegression(random_state=0, solver='sag').fit(X, Y)
#test_X = clf.predict(X)
false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('svm') 
print(clf1.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

#clf2 = RandomForestClassifier(random_state=1)
clf3 = LogisticRegression(penalty='l2', solver='lbfgs', random_state=1, class_weight='balanced')
#eclf = VotingClassifier(estimators=[('svc', clf1), ('lr', clf3)], voting='soft', weights=[2, 1])
#clf2.fit(X, Y)
clf3.fit(X_train_transformed, Y_train)
dump(clf3, 'trained_ev_detect_lr_30.joblib')
test_Y = clf3.predict(X_test_transformed)

#eclf.fit(X, Y)
#test_Y_el = eclf.predict(X)
false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('logregression') 
print(clf3.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)


clf2 = RandomForestClassifier(n_estimators=100, min_samples_split=6, min_samples_leaf=3, max_depth=50)

clf2.fit(X_train_transformed, Y_train)
dump(clf2, 'trained_ev_detect_rf_30.joblib')
test_Y = clf2.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('randomforest')     
print(clf2.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

dt_stump = DecisionTreeClassifier(max_depth=10, min_samples_leaf=6)
clf4 = AdaBoostClassifier(base_estimator=dt_stump)
clf4.fit(X_train_transformed, Y_train)
test_Y = clf4.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('adaboost') 
print(clf4.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

from sklearn.ensemble import GradientBoostingClassifier
clf5 = GradientBoostingClassifier(n_estimators=100, min_samples_split=6, min_samples_leaf=3, 
                                  max_depth=20, random_state=0)
clf5.fit(X_train_transformed, Y_train)
test_Y = clf5.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('gradientboost')  
print(clf5.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

from sklearn.neighbors import KNeighborsClassifier
clf6 = KNeighborsClassifier(n_neighbors=2)
clf6.fit(X_train_transformed, Y_train)
test_Y = clf6.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
print('KNN')  
print(clf5.score(X_test_transformed, Y_test))  
print(false_positive, true_negative, false_negative, true_positive)

eclf = VotingClassifier(estimators=[('svm', clf1), ('lr', clf3), ('rf', clf2)], 
                        voting='soft', weights=[1,2,1])
eclf.fit(X_train_transformed, Y_train)
test_Y = eclf.predict(X_test_transformed)

false_positive = 0
false_negative = 0
true_negative = 0
true_positive = 0
for i in range(0, len(Y_test)):
    if Y_test[i] != test_Y[i]:
        if Y_test[i] == 0:
            false_positive += 1
        elif Y_test[i] == 1:
            false_negative += 1
    else:
        if Y_test[i] == 0:
            true_negative += 1
        elif Y_test[i] == 1:
            true_positive += 1
            
print(false_positive, true_negative, false_negative, true_positive)
print(eclf.score(X_test_transformed, Y_test))  
score_proba = eclf.predict_proba(X_test_transformed)
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(Y_test, score_proba[:,1])
print(fpr)
print(tpr)
print(thresholds)
dump(eclf, 'trained_ev_detect_ensemble_15.joblib') 
dump(eclf, 'trained_ev_detect_ensemble_30.joblib') 

svm
0.7604166666666666
9 24 37 122
logregression
0.7708333333333334
12 21 32 127
randomforest
0.875
22 11 2 157
adaboost
0.84375
22 11 8 151
gradientboost
0.828125
20 13 13 146
KNN
0.828125
16 17 38 121
19 14 9 150
0.8541666666666666
[0.         0.         0.         0.03030303 0.03030303 0.06060606
 0.06060606 0.12121212 0.12121212 0.15151515 0.15151515 0.18181818
 0.18181818 0.21212121 0.21212121 0.24242424 0.24242424 0.27272727
 0.27272727 0.33333333 0.33333333 0.39393939 0.39393939 0.45454545
 0.45454545 0.51515152 0.51515152 0.54545455 0.54545455 0.57575758
 0.57575758 0.60606061 0.60606061 0.63636364 0.63636364 0.72727273
 0.72727273 1.        ]
[0.         0.00628931 0.20125786 0.20125786 0.27672956 0.27672956
 0.3081761  0.3081761  0.44025157 0.44025157 0.47169811 0.47169811
 0.55345912 0.55345912 0.62893082 0.62893082 0.72955975 0.72955975
 0.78616352 0.78616352 0.79874214 0.79874214 0.81761006 0.81761006
 0.82389937 0.82389937 0.8490566  0.8490566  0.93710692 0.93710692
 0.94

['trained_ev_detect_ensemble_30.joblib']